<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# AWS Elastic Map Reduce

---

## LEARNING OBJECTIVES

- Spin up a Spark cluster on AWS
- Run Spark jobs on the cluster
- Retrieve results from the hadoop file system (HDFS)

<h1>Lesson Guide<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#LEARNING-OBJECTIVES" data-toc-modified-id="LEARNING-OBJECTIVES-1">LEARNING OBJECTIVES</a></span></li><li><span><a href="#Intro-to-EMR" data-toc-modified-id="Intro-to-EMR-2">Intro to EMR</a></span></li><li><span><a href="#EMR-Pricing" data-toc-modified-id="EMR-Pricing-3">EMR Pricing</a></span></li><li><span><a href="#EMR-cluster" data-toc-modified-id="EMR-cluster-4">EMR cluster</a></span><ul class="toc-item"><li><span><a href="#Launch-Cluster" data-toc-modified-id="Launch-Cluster-4.1">Launch Cluster</a></span></li></ul></li><li><span><a href="#Accessing-the-EMR-cluster" data-toc-modified-id="Accessing-the-EMR-cluster-5">Accessing the EMR cluster</a></span></li><li><span><a href="#Configure-Web-Connection" data-toc-modified-id="Configure-Web-Connection-6">Configure Web Connection</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Enable-SSH-access-to-our-master-node." data-toc-modified-id="Enable-SSH-access-to-our-master-node.-6.0.1">Enable SSH access to our master node.</a></span></li><li><span><a href="#Install-and-configure-Foxy-Proxy-as-explained-here." data-toc-modified-id="Install-and-configure-Foxy-Proxy-as-explained-here.-6.0.2">Install and configure Foxy-Proxy as explained <a href="https://docs.aws.amazon.com/ElasticMapReduce/latest/ManagementGuide/emr-connect-master-node-proxy.html" target="_blank">here</a>.</a></span></li></ul></li></ul></li><li><span><a href="#Exercise" data-toc-modified-id="Exercise-7">Exercise</a></span></li><li><span><a href="#Hadoop" data-toc-modified-id="Hadoop-8">Hadoop</a></span><ul class="toc-item"><li><span><a href="#HDFS" data-toc-modified-id="HDFS-8.1">HDFS</a></span></li></ul></li><li><span><a href="#YARN" data-toc-modified-id="YARN-9">YARN</a></span></li><li><span><a href="#Exploring-HDFS-from-the-command-line" data-toc-modified-id="Exploring-HDFS-from-the-command-line-10">Exploring HDFS from the command line</a></span></li><li><span><a href="#Exploring-HDFS-from-the-web-interface" data-toc-modified-id="Exploring-HDFS-from-the-web-interface-11">Exploring HDFS from the web interface</a></span><ul class="toc-item"><li><span><a href="#Exercise" data-toc-modified-id="Exercise-11.1">Exercise</a></span></li></ul></li><li><span><a href="#Conclusions" data-toc-modified-id="Conclusions-12">Conclusions</a></span></li><li><span><a href="#ADDITIONAL-RESOURCES" data-toc-modified-id="ADDITIONAL-RESOURCES-13">ADDITIONAL RESOURCES</a></span></li></ul></div>

## Intro to EMR 

In a previous lesson we have discovered two very important AWS services: EC2 and S3. Today we will see how to spin up a computer cluster on Amazon. 

**What is a cluster?**

**What is a typical topology for a Big Data computing cluster?**

Amazon Elastic MapReduce was introduced in April 2009 to automate _provisioning_ of the Hadoop cluster, running and terminating jobs, and handling data transfer between EC2 (VM) and S3 (Object Storage). It simplifies the management of a Hadoop cluster, making it available to anyone at the click of a button.

EMR offers several pre-installed software packages including:

- Hadoop
- HBase
- Pig
- Hive
- Hue
- Spark
and many others.

## EMR Pricing

EMR Pricing is based on the type of instances forming the cluster and it's divided in tiers. The pricing adds to the cost of spinning up the instances in EC2.

Also, very importantly, costs are calculated in hourly increments, so if we plan to use the cluster for two sessions of half an hour, we should have it up for one hour consecutively instead of spinning it up and down twice.

EMR is not included in the AWS free tier that you've used in the previous class, so it's always a good practice to do some price checking before you spin up a cluster.

We can use the [AWS cost calculator](https://calculator.s3.amazonaws.com/index.html) to estimate the cost of a  three-node cluster with medium size instances `(m3.xlarge)`. The image below shows the cost for one hour: it's slightly more than one dollar.

![](./assets/images/emrcost.png)

If we were to keep the cluster alive for a month, that would result in a pretty high price, that's why it's so convenient to spin up and down clusters as they are needed.

## EMR cluster 

Let's spin up an EMR cluster with Spark and let's use it to perform a simple spark-ml gridsearch.

Let's first log-in to AWS and go to the EMR service page:

![](./assets/images/emr.png)

### Launch Cluster

![](./assets/images/clusterstart.png)

**Remember to choose the key pair you have already stored on your computer.**


![](./assets/images/clusterstarting.png)

**Notice also that like for EC2 we can list the clusters using the Cluster List pane:**

![](./assets/images/clusterlist.png)

**The cluster will take several minutes to boot completely. Press the circular refresh button in the top right of the console summary ("Cluster list") to refresh your view and see if the cluster is ready.**

**In the meantime, let's do a couple of review checks:**

---

**Do you remember how to connect to an instance on EC2?**

**Do you remember which commands we used in AWSCLI?**

**Do you remember how to transfer files from your computer to your bucket?**

---
**Once the cluster is ready we will see it in green:**

![](./assets/images/clusterready.png)

## Accessing the EMR cluster 

Go ahead and SSH to your master.
To do so, go to your ec2 console. You will see three new ec2 instances. Select the descriptions. You can see that one of the instances is the EMR-master and the other two are the EMR-slaves. 

Log in to the master using ssh in a similar format as a normal single ec2 instance (even though this is a cluster of three instances). Note that now you have to indicate **hadoop** instead of ec2-user:

```bash
$ ssh -i your_key_file.pem hadoop@your_public_DNS_for_the_master
```

> Note: If you cannot login, you have to add a security group for ssh with port 22.

Go to "Security Groups" under "Network & Security" on the left side-bar and add an inbound rule for:

SSH | TCP | 22 | Custom | 0.0.0.0/0

ensuring you select "Save". 

## Configure Web Connection 

EMR provides several user interfaces which we can access through the browser. 
Before we can do that, we will have to go trough one more step. In fact, the default security settings for EMR are pretty tight and do not allow for external web connections to our cluster. In order to connect with a browser we will have to set up an _ssh tunnel_ , i.e. have our browser communicate to the cluster via an encrypted channel. 

Luckily, Amazon provides us with simple instructions:

![](./assets/images/webconnection.png)

![](./assets/images/sshtunnel.png)

In order to follow them we first need to complete two steps:

#### Enable SSH access to our master node. 

This is done in the Security Groups pane of the EC2 services page.

![](./assets/images/securitygroups.png)

#### Install and configure Foxy-Proxy as explained [here](https://docs.aws.amazon.com/ElasticMapReduce/latest/ManagementGuide/emr-connect-master-node-proxy.html).

Once we have enabled SSH access, we can go ahead and connect:

```bash
ssh -i ~/.ssh/MyFirstKey.pem -ND 8157 hadoop@<YOUR_MASTER_DNS>
```

Note that this command will not end because it's keeping the tunnel alive.

If the tunnel and Foxy-proxy are well configured, we should be able to connect to several web services. 

You can use the foxy-proxy file found [here](foxy-proxy/foxyproxy-settings.xml).

## Exercise

- Load the spark-scripts to your EMR master node. 
- Load the data file to your S3 bucket.
- Run the Spark script with `spark-submit`. Don't forget to adjust the number of cores used for the grid search.


## Hadoop

---

Apache Hadoop is an open-source software framework for distributed storage and distributed processing of very large data sets on computer clusters built from commodity hardware. All the modules in Hadoop are designed with the fundamental assumption that hardware failures are common and should be automatically handled by the framework.

The core of Apache Hadoop consists of a storage part, known as **Hadoop Distributed File System (HDFS)**, and a processing part called MapReduce. Hadoop splits files into large blocks and distributes them across nodes in a cluster.

### HDFS

The Hadoop distributed file system (HDFS) is a distributed, scalable, and portable file-system written in Java for the Hadoop framework. It's the file system supporting Hadoop.

## YARN
---

Yarn is a resource-management platform responsible for managing computing resources in clusters and using them for scheduling of users' applications. The fundamental idea of YARN is to split up the functionalities of resource management and job scheduling/monitoring into separate daemons. The idea is to have a global ResourceManager (RM) and per-application ApplicationMaster (AM).

The ResourceManager and the NodeManager form the data-computation framework. The ResourceManager is the ultimate authority that arbitrates resources among all the applications in the system. The NodeManager is the per-machine framework agent who is responsible for containers, monitoring their resource usage (cpu, memory, disk, network) and reporting the same to the ResourceManager/Scheduler.

The Yarn resource manager offers a web interface that is accessible in our browser at this address:

    http://<YOUR_MASTER_DNS>:8088/

Go ahead and type that in your browser and you should see a screen like this:

![](./assets/images/yarn.png)

This will be useful when we run a hadoop job in order to check the status of advancement.

## Exploring HDFS from the command line
---

Hadoop offers a command line interface to navigate the HDFS. The full documentation can be found here:

http://hadoop.apache.org/docs/current/hadoop-project-dist/hadoop-common/FileSystemShell.html

You can explore the content of the HDFS with commands similar to those  used in the shell, for example

```bash
hadoop fs -ls
```

## Exploring HDFS from the web interface
---

Hadoop also offers a web interface to navigate and manage HDFS. It can be found at this address:

    http://<YOUR_MASTER_DNS>:50070
    

and it looks like this:

![](./assets/images/hdfsweb.png)


Click [here](http://docs.aws.amazon.com/emr/latest/ManagementGuide/emr-web-interfaces.html) for further browser interfaces like yarn.

### Exercise 
Find how you can navigate the HDFS from the web interface. Is the content listed similar to what you were finding with the command line?


You can copy results from the HDFS to your master node or the other way round indicating the file name (replace `filename` appropriately) 


```hadoop
hadoop fs -copyToLocal filename filename
hadoop fs -copyFromLocal filename filename
```

## Conclusions

We have learned how to spin up a cluster on AWS with spark pre-installed.

**Make sure you terminate your cluster now:**

![](./assets/images/terminate.png)

**Delete the buckets from S3, to avoid paying for storage space.**

![](./assets/images/deletebucket.png)


> Now that you're enabled with the ability to process very large datasets in the cloud, what problems would you like to tackle?

## ADDITIONAL RESOURCES

- [AWS EMR tutorial](http://docs.aws.amazon.com//ElasticMapReduce/latest/ManagementGuide/emr-gs.html)